In [51]:
# Imports
import tkinter as tk
import os
import platform
import glob
from tkinter import filedialog
import re
from pathlib import Path
import shutil
## Sets backend plotting for matplotlib to use notebook so that it is interactable. This is specifically for PyCharm so you can comment it out if you don't need it
%matplotlib notebook 

In [52]:
# Set up all file directories
## Create a root window and immediately hide it
root = tk.Tk()
root.withdraw()

## Open a directory selector UI for raw data location
### e.g. D:/Data/Preprocessing/Raw/Orange/2023-08-29_15-17-41
dir_path = filedialog.askdirectory(title="Please select a data folder")
### Check if user selected a directory or pressed cancel
if dir_path:
    print(f"Selected Data Directory: {dir_path}")
else:
    print("No directory was selected.")

## Open a directory selector UI for output directory.
output_path = filedialog.askdirectory(title="Please select a output folder")
if dir_path:
    print(f"Selected Data Directory: {output_path}")
else:
    print("No directory was selected.")


## Add in the correct locations for NeuroPy repositories on your computer(s) here:
if platform.system() == "Linux":
    print("Detected Linux OS")
elif platform.system() == "Darwin":
    print("Detected MacOS")
elif platform.system() == "Windows":
    print("Detected Windows OS")
else:
    print("Unknown OS.") # This should never show. Big issues if this happens I guess.
    
## Set up output directory, and if it doesn't exist, create it.
base_path = Path(dir_path)
output_path = Path(output_path)
date_folder = base_path.name.split('_')[0]
new_output_path = output_path / date_folder
new_output_path.mkdir(exist_ok=True)
date_part = dir_path.split('/')[-1].split('_')[0]
formatted_date = date_part.replace('-', '')
print(f"New folder created at: {new_output_path}")

Selected Data Directory: /home/kimqi/Documents/Data/Raw/Orange/2023-08-17_10-01-16
Selected Data Directory: /home/kimqi/Documents/Data/Preprocessed Data/Orange
Detected Linux OS
New folder created at: /home/kimqi/Documents/Data/Preprocessed Data/Orange/2023-08-17


In [53]:
# Get record node from user.
entries = os.listdir(dir_path)
recordn_dir = [entry for entry in entries if os.path.isdir(os.path.join(dir_path, entry))]
nodes = [int(re.search(r'\d+', node).group()) for node in recordn_dir]

# Display choices to user
print("Select a record node for preprocessing:")
for i, num in enumerate(nodes, 1):
    print(f"{i}. {num}")

#
choice = int(input("Enter the choice number: "))

# Validate choice
if choice in nodes:
    selected_record_node = choice
    print(f"Preprocessing {selected_record_node}")
else:
    print("Invalid choice. Stopping preprocessing step.")

Select a record node for preprocessing:
1. 106
Preprocessing 106


In [54]:
# Find and print files associated with the selected record node, check to see if all the record nodes have been represented.
if platform.system() == "Windows":
    pattern = f"Record Node {selected_record_node}\\**\\*continuous.dat"
    for file in base_path.glob(pattern):
        print(file)
elif platform.system() == "Linux":
    pattern = f"Record Node {selected_record_node}/**/*continuous.dat"
    for file in base_path.glob(pattern):
        print(file)

/home/kimqi/Documents/Data/Raw/Orange/2023-08-17_10-01-16/Record Node 106/experiment2/recording1/continuous/Intan_Rec._Controller-100.0/continuous.dat
/home/kimqi/Documents/Data/Raw/Orange/2023-08-17_10-01-16/Record Node 106/experiment2/recording3/continuous/Intan_Rec._Controller-100.0/continuous.dat
/home/kimqi/Documents/Data/Raw/Orange/2023-08-17_10-01-16/Record Node 106/experiment2/recording2/continuous/Intan_Rec._Controller-100.0/continuous.dat
/home/kimqi/Documents/Data/Raw/Orange/2023-08-17_10-01-16/Record Node 106/experiment1/recording1/continuous/Intan_Rec._Controller-100.0/continuous.dat


In [55]:
# Concatenate recording files together
files_to_concatenate = sorted(base_path.glob(pattern))
print(files_to_concatenate)

# Print the files to be concatenated
print("Concatenating the following files:")
for f in files_to_concatenate:
    print(f)

# Concatenate the files
dat_file = base_path / "continuous_combined.dat"
with dat_file.open('wb') as wfd:
    for f in files_to_concatenate:
        with f.open('rb') as fd:
            shutil.copyfileobj(fd, wfd)
eeg_file = dat_file.with_suffix(".eeg")

[PosixPath('/home/kimqi/Documents/Data/Raw/Orange/2023-08-17_10-01-16/Record Node 106/experiment1/recording1/continuous/Intan_Rec._Controller-100.0/continuous.dat'), PosixPath('/home/kimqi/Documents/Data/Raw/Orange/2023-08-17_10-01-16/Record Node 106/experiment2/recording1/continuous/Intan_Rec._Controller-100.0/continuous.dat'), PosixPath('/home/kimqi/Documents/Data/Raw/Orange/2023-08-17_10-01-16/Record Node 106/experiment2/recording2/continuous/Intan_Rec._Controller-100.0/continuous.dat'), PosixPath('/home/kimqi/Documents/Data/Raw/Orange/2023-08-17_10-01-16/Record Node 106/experiment2/recording3/continuous/Intan_Rec._Controller-100.0/continuous.dat')]
Concatenating the following files:
/home/kimqi/Documents/Data/Raw/Orange/2023-08-17_10-01-16/Record Node 106/experiment1/recording1/continuous/Intan_Rec._Controller-100.0/continuous.dat
/home/kimqi/Documents/Data/Raw/Orange/2023-08-17_10-01-16/Record Node 106/experiment2/recording1/continuous/Intan_Rec._Controller-100.0/continuous.dat
/h

In [57]:
!process_resample -n 35 -f 30000,1250 {dat_file} {eeg_file}

Input File     : /home/kimqi/Documents/Data/Raw/Orange/2023-08-17_10-01-16/continuous_combined.dat
Channels       : 35
Sampling Rate  : 30000.000000
Output file    : /home/kimqi/Documents/Data/Raw/Orange/2023-08-17_10-01-16/continuous_combined.eeg
Sampling Rate  : 1250.000000
Freq Ratio     : 0.041667
Converter      : Fastest Sinc Interpolator
Resampling     : 0% ##################################################
Output Frames  : 1515008


In [58]:
## Copy the .eeg file from original raw folder to preprocessed folder.
import shutil
from pathlib import Path

# Specify the source file and target directory
src_file = Path(eeg_file)
target_dir = Path(new_output_path)

new_file_name = formatted_date+".eeg"

# Combine the target directory with the new file name
dst_file = target_dir / new_file_name

# Copy and rename
shutil.copy(src_file, dst_file)

print(f"Copied and renamed file to: {dst_file}")

Copied and renamed file to: /home/kimqi/Documents/Data/Preprocessed Data/Orange/2023-08-17/20230817.eeg
